# TensorFlow スクリプトモードによる学習とモデルホスティング

スクリプトモードは、TensorFlowのトレーニングスクリプト形式で、SageMakerでTensorFlowトレーニングスクリプトを最小限の変更で実行できます。 [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) は、SageMakerトレーニングインスタンスへのスクリプトの転送を処理します。トレーニングインスタンスでは、SageMakerのネイティブTensorFlowサポートがトレーニング関連の環境変数を設定し、トレーニングスクリプトを実行します。このチュートリアルでは、SageMaker Python SDKを使用してトレーニングジョブを起動し、トレーニングされたモデルを展開します。

スクリプトモードは、Pythonスクリプト、Pythonモジュール、またはシェルスクリプトによるトレーニングをサポートします。この例では、Pythonスクリプトを使用して、[MNISTデータセット](http://yann.lecun.com/exdb/mnist/)の分類モデルをトレーニングします。さらに、このノートブックは、[SageMaker TensorFlow Serving container](https://github.com/aws/sagemaker-tensorflow-serving-container)でリアルタイム推論を実行する方法を示します。 TensorFlow Servingコンテナは、スクリプトモードのデフォルトの推論方法です。 TensorFlow Servingコンテナの詳細なドキュメントについては、[こちら](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst) にアクセスしてください。


# 1. 環境のセットアップ

まずは環境のセットアップを行いましょう。

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

print('Current SageMaker Python sdk Version ={0}'.format(sagemaker.__version__))

注） Spot インスタンスを使った学習を行う際は、SageMaker Python SDK のバージョン が 1.37.2 以上である必要があります。上記の出力結果がそれ以前のバージョンになった際は、下記を実行し、Jupyterカーネルを再起動し、再度上記のセルを実行し、バージョンがアップデートされたことを確認してください。カーネルが再起動されない場合は、SageMaker SDK バージョン更新が反映されません。

In [ ]:
!pip install -U --quiet "sagemaker>=1.37.2"

# 2. 学習データの準備

MNISTデータセットは、パブリックS3バケット ``sagemaker-sample-data-<REGION>`` の下のプレフィックス ``tensorflow/mnist`` の下にロードされています。 このプレフィックスの下には4つの ``.npy`` ファイルがあります：
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [ ]:
training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)

In [ ]:
!wget s3://sagemaker-sample-data-us-west-2/tensorflow/mnist

# 3. 分散学習用のスクリプトを作成する

このチュートリアルのトレーニングスクリプトは、TensorFlowの公式の[CNN MNISTの例](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/layers/cnn_mnist.py) をベースに作成されました。 SageMaker から渡された `` model_dir`` パラメーターを処理するように変更しています。 これは、分散学習時のデータ共有、チェックポイント、モデルの永続保存などに使用できるS3パスです。 また、トレーニング関連の変数を扱うために、引数をパースする関数も追加しました。

トレーニングジョブの最後に、トレーニング済みモデルを環境変数 ``SM_MODEL_DIR`` に保存されているパスにエクスポートするステップを追加しました。このパスは常に ``/opt/ml/model`` をポイントします。 SageMaker は、トレーニングの終了時にこのフォルダー内のすべてのモデル成果物をS3にアップロードするため、これは重要です。

スクリプト全体は次のとおりです。

In [ ]:
!pygmentize 'mnist.py'

# 4. TensorFlow Estimator を利用して学習ジョブを作成する

`sagemaker.tensorflow.TensorFlow`　estimator は、スクリプトモード対応の TensorFlow コンテナの指定、学習・推論スクリプトの S3 へのアップロード、および SageMaker トレーニングジョブの作成を行います。ここでいくつかの重要なパラメーターを呼び出しましょう。

* `py_version`は` 'py3'`に設定されています。レガシーモードは Python 2 のみをサポートしているため、この学習スクリプトはスクリプトモードを使用していることを示しています。Python2は間もなく廃止されますが、 `py_version` を設定することでPython 2でスクリプトモードを使用できます。`'py2'`と` script_mode`を `True`にします。

* `distributions` は、分散トレーニング設定を構成するために使用されます。インスタンスのクラスターまたは複数の GPU をまたいで分散学習を行う場合にのみ必要です。ここでは、分散トレーニングスキーマとしてパラメーターサーバーを使用しています。 SageMaker トレーニングジョブは同種のクラスターで実行されます。 SageMaker セットアップでパラメーターサーバーのパフォーマンスを向上させるために、クラスター内のすべてのインスタンスでパラメーターサーバーを実行するため、起動するパラメーターサーバーの数を指定する必要はありません。スクリプトモードは、[Horovod](https://github.com/horovod/horovod) による分散トレーニングもサポートしています。 `distributions` の設定方法に関する詳細なドキュメントは[こちら](https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/tensorflow#distributed-training) をご参照ください。

Spot Instanceを用いて実行する場合は、下記のコードを `Estimator` の `train_instance_type` の次の行に追加しましょう。

```python
                             train_max_run = 5000,
                             train_use_spot_instances = 'True',
                             train_max_wait = 10000,
```


In [ ]:
from sagemaker.tensorflow import TensorFlow


mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             train_instance_count=2,
                             train_instance_type='ml.p2.xlarge',
                             framework_version='1.14',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': True}})

## ``fit`` による学習ジョブの実行

学習ジョブを開始するには、`estimator.fit（training_data_uri）` を呼び出します。

ここでは、S3 ロケーションが入力として使用されます。 `fit` は、`training` という名前のデフォルトチャネルを作成します。これは、このS3ロケーションを指します。トレーニングスクリプトでは、 `SM_CHANNEL_TRAINING` に保存されている場所からトレーニングデータにアクセスできます。 `fit`は、他のいくつかのタイプの入力も受け入れます。詳細については、APIドキュメント[こちら](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit) を参照してください。

トレーニングが開始されると、TensorFlow コンテナは mnist.py を実行し、スクリプトの引数として　estimator から`hyperparameters` と `model_dir` を渡します。この例では、estimator 内で定義していないハイパーパラメーターは渡されず、 `model_dir` のデフォルトは `s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>` であるため、スクリプトの実行は次のようになります。
```bash
python mnist.py --model_dir s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>
```
トレーニングが完了すると、トレーニングジョブは保存されたモデルを TensorFlow serving にアップロードします。

In [ ]:
mnist_estimator.fit(training_data_uri)

# 5. 学習したモデルをエンドポイントにデプロイする

`deploy（）`メソッドは SageMaker モデルを作成します。このモデルはエンドポイントにデプロイされ、リアルタイムで予測リクエストを処理します。 スクリプトモードでトレーニングしたため、エンドポイントには TensorFlow Serving コンテナを使用します。 このサービングコンテナは、SageMaker ホスティングプロトコルと互換性のあるWebサーバーの実装を実行します。 [独自の推論コードの使用](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/your-algorithms-inference-main.html) ドキュメントでは、SageMaker が推論コンテナを実行する方法について説明しています。

In [ ]:
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# 6. エンドポイントを呼び出し推論を実行する

トレーニングデータをダウンロードして、推論の入力として使用してみましょう。

入力データと出力データの形式は、[TensorFlow Serving REST API](https://www.tensorflow.org/serving/api_rest) の `Predict`メソッドのリクエストとレスポンスの形式に直接対応しています。 SageMaker の TensforFlow Serving エンドポイントは、単純化されたJSON形式、行区切りのJSONオブジェクト ("jsons" または "jsonlines")、CSV データなど、TensorFlow REST API の一部ではない追加の入力形式も受け入れることができます。

この例では、入力として `numpy` 配列を使用しています。これは、簡略化されたJSON形式にシリアル化されます。 さらに、TensorFlow serving は、次のコードに示すように、複数のアイテムを一度に処理することもできます。 TensorFlow serving を用いた SageMaker エンドポイントに対して予測を行う方法に関する詳細なドキュメントは[こちら](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#making-predictions-against-a-sagemaker-endpoint)をご参照ください。

まず、評価用のデータセットを取得します。

In [ ]:
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/eval_data.npy eval_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/eval_labels.npy eval_labels.npy

eval_data = np.load('eval_data.npy')
eval_labels = np.load('eval_labels.npy')

下記の ``k　= `` に、9950 までの好きな数字を入れて、評価する手書き文字のセットを選択しましょう。
`predictor.predict(test_data)` で推論を行い、選んだ手書き文字認識に対する `prediction is`: 推論結果 と、`label is`: 実際のラベルの値が出力され、その二つが一致していれば最後に `matched: True` と表示されます。

In [ ]:
k = 1000 # choose your favorite number from 0 to 9950
test_data = eval_data[k:k+50]
test_data
import matplotlib.pyplot as plt
for i in range(5):
    for j in range(10):
        plt.subplot(5, 10, 10* i + j+1)
        plt.imshow(test_data[10 * i + j, :].reshape(28, 28), cmap='gray')
        plt.title(10* i + j+1)
        plt.tick_params(labelbottom=False, labelleft = False)
        plt.subplots_adjust(wspace=0.2, hspace=1)
plt.show()

In [ ]:
predictions = predictor.predict(test_data)
for i in range(0, 50):
    prediction = predictions['predictions'][i]['classes']
    label = eval_labels[i+k]
    print(' [{}]: prediction is {}, label is {}, matched: {}'.format(i+1, prediction, label, prediction == label))

# 7. エンドポイントを削除する

余分なコストが発生しないように、検証が終わったら上記で作成したエンドポイントを削除しましょう。

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)